## Step 1: Install Python packages

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests
import selenium
from selenium import webdriver                   
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pymongo import MongoClient

## Step 2: Preparation

In [2]:
#Initialize browser in current directory
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)
driver = webdriver.Firefox(executable_path= "geckodriver.exe")
driver.maximize_window()


In [3]:
news="https://mars.nasa.gov/news/"
feature="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
facts="https://space-facts.com/mars/"
hemisphere="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

<div class="alert alert-block alert-info">
    <b>Note:</b> The next lines of code is the connection to a mongo databaste which will be seperated into different collections through this document.
</div>

In [4]:
# Connect to MongoDB
#connecting to the local host
client =  MongoClient("mongodb://localhost:27017")
#creating database
db = client['mars_database']

## Step 3: Find the IDs of the items we want to scrape for

In [5]:
driver.get(news)

In [6]:
# news_page_id=browser.find_by_id('site_body')
timeout = 20
# Find an ID on the page and wait before executing anything until found: 
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "site_body")))
except TimeoutException:
    driver.quit()

In [ ]:
#news_page=driver.find_element_by_id("site_body").get_attribute("innerHTML")
# body = driver.find_element_by_tag_name("body")
# news_page = body.get_attribute("innerHMTL")

In [ ]:
#news_soup = BS(news_page,'html.parser')

In [8]:
# Retrieve the latest news title
title_elements = driver.find_elements_by_class_name("content_title")
title_htmls = [title_element.get_attribute("innerHTML") for title_element in title_elements]
title_html = title_htmls[1]
news_soup = BS(title_html, 'lxml')
news_title = news_soup.get_text()
news_title

'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth'

In [9]:
teaser_element = driver.find_element_by_class_name("article_teaser_body")
news_p = teaser_element.get_attribute("innerHTML")
news_p

"Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."

In [10]:
driver.get(feature)

In [11]:
timeout = 20
# Find an ID on the page and wait before executing anything until found: 
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "site_body")))
except TimeoutException:
    driver.quit()

In [12]:
#LOOK FOR AN ID FIRST, THEN FIND THE ATTRIBUTE
feature_image = driver.find_element_by_id("full_image")
# featured_image = [image_element.get_attribute("data-link") for image_element in feature_image]
# image_link=feature+featured_image[0]
# driver.get(image_link)
feature_image.click()


In [13]:
feature_image_next = driver.find_element_by_link_text("more info")
feature_image_next.click()
# featured_image_next = [image_element.get_attribute("data-link") for image_element in feature_image_next]
# image_link_forrealz=feature+featured_image_next[0]
# image_link_forrealz
# driver.get(image_link_forrealz)

In [14]:
# featured_soup = BeautifulSoup(html, 'html.parser')

#main_image= featured_soup.find('img', class_='main_image')

large_image = driver.find_elements_by_class_name("main_image")

feat_images=[]
for feat_image in large_image:
    feat_images.append(feat_image.get_attribute("src"))

featured_image_url=feat_images[0]
# featured_image_url = feat_image
# main_image
    
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18182_hires.jpg'

In [ ]:
tables = pd.read_html(facts)
mars_table=tables[0]

In [ ]:
driver.get(hemisphere)

In [15]:
hemisphere_images = driver.find_element_by_id("product-section")
hemi_titles = [hemi_title.get_attribute("h3") for hemi_title in hemisphere_images]
hemi_titles
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]

WebDriverException: Message: Failed to write request to stream


In [ ]:
browser.quit()

## Step 4: The full code that runs the scraper and save the data to mongo db

In [ ]:
# news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

# news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."

# featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

# mars_table

# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]

In [ ]:
#This is ref./example to understand that this is the same thing.
#mars_database.mars_scrape=mars_database["mars_scrape"]

# A dictionary that will become a MongoDB document
mars_dictionary = {
    'title': news_title,
    'paragraph': news_p,
    'image': featured_image_url,
    'table': mars_table,
    'urls': hemisphere_image_urls
}
# Declare the collection
mars_collection = db.mars_scrape
# Insert document into collection
mars_collection.insert_one(mars_dictionary)
